## Data Exploration of Los Angeles Police Stations 
With the given data on by the LAPD, I will be exploring where police stations are located in LA. Doing so while running multiple commands to get more comfortable with data exploration features. The end goal, however, is for me to make a map that marks the closest LAPD station and the UCLA campus. Just in case anyone needs LAPD instead of campus police, you never know!

First things first, I import the high level geospatial library, `geopandas`.

In [ ]:
import geopandas as gpd

The next big step is to import data from the LAPD on their police stations in the county.

In [ ]:
policeStations = gpd.read_file('data/LAPD_policestation.shp')

My first view of this data is to see how many columns and rows it holds, so I know how much data there is for me to work with. I do this with the `.shape` command.

In [ ]:
policeStations.shape

To see the first five stations on the dataset, I use the `.head()`command.

In [ ]:
policeStations.head()

And for the last fifve stations, I use the `.tail()` command.

In [ ]:
policeStations.tail()

The `.tail()` command reveals that the data accounts for the 20 LAPD station locations.

To see the specific features of the 21 stations, I next utilize the `.info()` command.

In [ ]:
policeStations.info()

This reveals the aspects of the data I can explore and their data types. Moreover, it shows that every station holds the same value (21). 

**Most** notably, is the `geometry` datatype. Sadly, this is the latitude and longitude location of each station. Which were not given seperately. Therefore, I have to exract those values myself.

This was done with help of this beautiful [slack overflow](https://stackoverflow.com/questions/49635436/shapely-point-geometry-in-geopandas-df-to-lat-lon-columns) post.

In [ ]:
policeStations['long'] = policeStations['geometry'].x
policeStations['lat'] = policeStations['geometry'].y

Now when I print the dataset, there should be an added `lat` and `long` columns to the end.

In [ ]:
policeStations

Successfully done, now the latitude and longitude columns are easily distinguished.

For this dataset, I want to see where the stations are and their relative location to eachother. Before mapping it out, I want to find out if any of the station happen to be the same location. This is not assumed, as police stations are not supposed to be, however it is Los Angeles and you *never* know. This is done with the `[' '].value_counts()` command.

In [ ]:
policeStations['LOCATION'].value_counts()

As assumed, every location only has one location listed. It is always good to confirm data!

Next, I want to see the plot to view the rough map of LA's police stations. Doing so with the `.plot()` command. 

In [ ]:
policeStations.plot()

Just off this rough plot, I can see that one station is WAY down there in the count compared to the others somewhat clustered together. I want to find out which this is, initially by creating diffferent colors of the plots.

In [ ]:
policeStations.plot(
            figsize=(10,10),
            column = 'DIVISION',  
            legend = True,         
            legend_kwds={
               'loc': 'upper right',
               'bbox_to_anchor':(1.3,1)
            }                  
) 

The code above came wtih help from out Python and Metro Juypter book!

Now, I want to trim the dataset and isolate the `lat`, `long` and `DIVISION` columns, so it is easier for me to read among the other columns and I can identify the location of the police divison.

In [ ]:
desiredColumns = ['DIVISION', 'lat', 'long']

policeStations[desiredColumns].head()

Since the dataset was already viewed to be in descending order, I used the `.head()` command to save space/time in order to easily identify that the Harbor `DIVISION` is the outliner location base on its small latitude and its plot location on the second plot.

For aesthetic purposes, I am going to rename the colums so they are not in different cases (UPPER v. lower).

In [ ]:
policeLocations = policeStations[desiredColumns]
policeLocations.columns.to_list()

This just shows the names as of now, while assigning them to a new variable dataset to work with. Next is the actual renaming process:

In [ ]:
policeLocations.columns = ['division', 'lat', 'long']
policeLocations

This allows for easier readablity of the data.

Though it is a smaller dataset, I want to see if there is a LAPD station in Westwood because that's where I live and it's important for college students (asside from campus police of course).

In [ ]:
policeLocations[policeLocations.division == 'WESTWOOD']

No luck! There are no LAPD Stations in Westwood but I am sure they is one close enough on Wilshire:

In [ ]:
policeLocations[policeLocations.division == 'WEST LOS ANGELES']

There it is! Now I know there is a station pretty close to UCLA in case anyone ever needs it (hopefully not though). 

Next I will make this station its own variable so I can compare it to UCLA and its respective latitude and longitude coordinates.

In [ ]:
westLA = policeLocations[policeLocations.division == 'WEST LOS ANGELES']
westLA

The coordinates from UCLA are: 34.06999972, -118.439789907. I change  the `wilshireStation` variable to be its own dataframe.

In [ ]:
westLA = westLA.reset_index()
westLA

In [ ]:
type(westLA)

This was to test and make sure that the datatype was in fact changed to a dataframe.

Next I have to eliminate the `index` column, as I have no need for it when I will be using only 2 locations. Using `.copy()` to insure it is saved seperately so there is no duplicate values.

In [ ]:
neededColumns = ['division', 'lat', 'long']
westLA = westLA[neededColumns].copy()

westLA

Now, I add UCLA and its coordinates (recieved off of google) to this new dataframe, achieved by using another helpful [link](https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/) with the ` .loc` command. 

In [ ]:
westLA.loc[len(westLA.index)] = ['UCLA', 34.06999972,  -118.439789907]  
  
westLA

Now we have a simple, newly made dataframe with Wilshire Police Station and UCLA.

Let's make it a map so we can visualize how close it is to campus though! First by importing `folium`

In [ ]:
import folium

Now I am going to find the mean `lat` and `long` so that I can get a feel for how big the map wil be to fit both markers.

In [ ]:
latitude = westLA.lat.mean()
latitude

In [ ]:
longitude = westLA.long.mean()
longitude

I am going to test out the `.map` command to see if the lat and long values are correctly located in LA.

In [ ]:
m = folium.Map(location=[latitude, longitude], zoom_start=14)
m

Now that I can see it is properly in West LA, including Westwood, I will add markers so the West LA police station and UCLA are marked. 

In [ ]:
for index, row in westLA.iterrows():
      folium.Marker(
    [row.lat, row.long], popup=row.division, tooltip=row.division
).add_to(m)
m

Now I am going to make the markers different colors! I will change the UCLA one to green and the police station to red.

In [ ]:
westLA['color'] = ''

In [ ]:
westLA.loc[westLA['division'] == 'WEST LOS ANGELES', 'color'] = 'red'
westLA.loc[westLA['division'] == 'UCLA', 'color'] = 'green'

westLA

Now that we know it worked, we can print the map out again. Hopefully with new colored markers!

In [ ]:
# add the stations with color icons
for index, row in westLA.iterrows():
    tooltip_text = row.division
    folium.Marker(
        [row.lat,row.long], 
        popup=row.division, 
        tooltip=tooltip_text,
        icon=folium.Icon(color=row.color)
    ).add_to(m)

# show the map
m

Done! Successfully explored the LAPD data while finishing having a map off the closet station to UCLA campus!